In [4]:
import pandas as pd
import snowflake.connector
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.subplots as sp
import plotly.io as pio
import os
from datetime import date, datetime, timedelta
import gdown

## PDF INFO

pdf_folder_name = 'events_pdfs'
folder_path = os.path.join(os.getcwd(),pdf_folder_name)
if not os.path.exists(folder_path):
    os.mkdir(folder_path)

## COLUMN CONSTANTS

# EVENT TABLE COLS - fill with "" if not applicable
EVENT_DATE_COL = 'event_date'
CUST_ID_COL = 'customer_id'
EVENT_TYPE_COL = 'event_type'
EVENT_DESC_COL = 'event_description'
EVENT_ID_COL = 'event_id'

# CUSTOMER INFO TABLE - fill with "" if not applicable
AGE_COL = 'age'
GENDER_COL = 'gender'
START_D_COL = 'start_date'
LOCATION_COL = 'location'
SALARY_COL = 'salary'

splitting_cols_numeric = [AGE_COL,SALARY_COL,START_D_COL]
splitting_cols_cat = [AGE_COL,GENDER_COL,LOCATION_COL]
time_intervals = ['month','week','day_of_week'] 

# Pull data, want 1,000,000 random rows
limit_num = 1000000

file_id = "1NsuIREETvfjoFzg3943MOB5Vr2Yc3hHR"
output_file = "full_events_data.pkl"

url = f"https://drive.google.com/uc?id={file_id}"
gdown.download(url, output_file)



Snowflake (old method)

In [ ]:
'''
## SNOWFLAKE INFO
account = 'or10868.uae-north.azure'
username = 'insait'
password = 'Insait123'
database = 'FAKEDB'
schema = 'EDA116'
warehouse = 'COMPUTE_WH'
role ='ACCOUNTADMIN'

table = 'event_logs_full_with_dep'
customer_info = 'customer_info'



# Snowflake connection
conn = snowflake.connector.connect(
    user=username,
    password=password,
    account=account,
    warehouse=warehouse,
    database=database,
    schema=schema
)

# Create a cursor object
cursor = conn.cursor()



# Get event log tables, put them into one data frame
cursor.execute(f"SELECT * FROM {schema}.{table} order by RANDOM(42) LIMIT "+str(limit_num))

data = cursor.fetchall()

# CLIENT TO DO: order the columns list properly to fit table structure
df = pd.DataFrame(data=data, columns = [CUST_ID_COL, EVENT_TYPE_COL, EVENT_ID_COL, EVENT_DATE_COL, 
                                        EVENT_DESC_COL])
df.sort_values(by=EVENT_DATE_COL, inplace=True)

# create more accessible date columns given the datetime object
df['year'] = df[EVENT_DATE_COL].dt.year
df['quarter'] = df[EVENT_DATE_COL].dt.quarter

df['month'] = df[EVENT_DATE_COL].dt.month
df['week'] = df[EVENT_DATE_COL].dt.isocalendar().week.astype(int)
df['day'] = df[EVENT_DATE_COL].dt.day
df['hour'] = df[EVENT_DATE_COL].dt.hour
df['day_of_week'] = df[EVENT_DATE_COL].dt.dayofweek 

# Option to pickle the dataframe for efficient access
# df.to_pickle("./dfpdf.pkl")

# Customer info
cursor.execute(f"SELECT * FROM {schema}.{customer_info}")
data = cursor.fetchall()

# CLIENT TO DO: order the columns list properly to fit table structure - add more as needed
df_cust_info = pd.DataFrame(data=data, columns = [CUST_ID_COL, AGE_COL, GENDER_COL, START_D_COL, LOCATION_COL, SALARY_COL])

# Option to pickle the dataframe for efficient access
# df_cust_info.to_pickle('./dfcustpdf.pkl')


# Read the pickles, if applicable
# df = pd.read_pickle('./dfpdf.pkl')
# df_cust_info = pd.read_pickle('./dfcustpdf.pkl')
'''

In [ ]:
# Pull data, want 1,000,000 random rows
limit_num = 1000000

df = pd.read_pickle('./' + output_file)

if EVENT_DESC_COL == '':
    df[EVENT_DESC_COL] = pd.Series(df[EVENT_TYPE_COL])

df_event = df[df['row_category'] == 'event']
df_event[EVENT_DATE_COL] = pd.to_datetime(df_event[EVENT_DATE_COL])
df_cust_info = df[df['row_category'] == 'cust_info']

# create more accessible date columns given the datetime object
df_event['year'] = df_event[EVENT_DATE_COL].dt.year
df_event['quarter'] = df_event[EVENT_DATE_COL].dt.quarter

df_event['month'] = df_event[EVENT_DATE_COL].dt.month
df_event['week'] = df_event[EVENT_DATE_COL].dt.isocalendar().week.astype(int)
df_event['day'] = df_event[EVENT_DATE_COL].dt.day
df_event['hour'] = df_event[EVENT_DATE_COL].dt.hour
df_event['day_of_week'] = df_event[EVENT_DATE_COL].dt.dayofweek 

df_randomized = df_event.sample(frac=1, random_state=42)  # Set random_state for reproducibility

# Take the first 1000 rows
df = df_randomized.head(limit_num)

df.sort_values(by=EVENT_DATE_COL, inplace=True)

# CLIENT TO DO: order the columns list properly to fit table structure - add more as needed\
splitting_cols_numeric = [AGE_COL,SALARY_COL,START_D_COL]
splitting_cols_cat = [AGE_COL,GENDER_COL,LOCATION_COL]

In [ ]:
customers = list(df[CUST_ID_COL].unique())
events = list(df[EVENT_TYPE_COL].unique())

In [5]:
## PLOTS

# Barplot
plot_name = 'event_frequency'
plot_path = os.path.join(folder_path,plot_name)

# Check if the folder already exists before creating it
if not os.path.exists(plot_path):
    os.mkdir(plot_path)
    
for event in ['All events'] + events:

    event_path = os.path.join(plot_path, event)

    # Check if the folder already exists before creating it
    if not os.path.exists(event_path):
        os.mkdir(event_path)

    for time_interval in time_intervals:
        for dist in [None] + splitting_cols_cat:

            dfc = df.copy()
            
            # For the given time interval, retreive the unique choices and assign them as the x axis values
            if time_interval == 'day_of_week':
                x_data = [i for i in dfc[time_interval].unique()]
            
            else:
                start_date = dfc[EVENT_DATE_COL].min()
                
                if time_interval == 'quarter':
                    dfc[time_interval] = dfc[EVENT_DATE_COL].apply(lambda x: (x.year - start_date.year) * 4 + (((x.month - 1) // 3 + 1) - ((start_date.month - 1) // 3 + 1)))
                
                elif time_interval == 'month':
                    dfc[time_interval] = dfc[EVENT_DATE_COL].apply(lambda x: (x.year - start_date.year) * 12 + (x.month - start_date.month))
                
                elif time_interval == 'week':
                    dfc[time_interval] = dfc[EVENT_DATE_COL].apply(lambda x: (x - start_date).days // 7)
                
                elif time_interval == 'day':
                    dfc[time_interval] = dfc[EVENT_DATE_COL].apply(lambda x: (x - start_date).days)
                
                
                x_data = [i for i in dfc[time_interval].unique()]
            
            x_data.sort()
            
            if dist: #show gender distribution
                dfcust = df_cust_info.copy()
                
                if dist == AGE_COL:
                    
                # Create an additional column of age ranges
                    bin_size = 10
                    min_range = (dfcust[AGE_COL].astype(int).min() // bin_size) * bin_size if dfcust[AGE_COL].astype(int).min() % bin_size != 0 else ((dfcust[AGE_COL].astype(int).min() // bin_size) - 1) * bin_size 
                    max_range = ((dfcust[AGE_COL].astype(int).max() // bin_size) + 1) * bin_size if dfcust[AGE_COL].astype(int).max() % bin_size != 0 else dfcust[AGE_COL].astype(int).max()
                    age_ranges = list(range(min_range, max_range + 1, bin_size))

                    dfcust['age_group'] = pd.cut(dfcust[AGE_COL].astype(int), bins=age_ranges)
                    dfcust['age_group'] = dfcust['age_group'].astype(str)

                    # Extract start values using string manipulation
                    dfcust['age_group'] = dfcust['age_group'].str.extract(r'\((.*),').astype(int)
                    
                    dist = 'age_group'
                
                
                    
                dfc = pd.merge(dfc, dfcust[[CUST_ID_COL, dist]], left_on=CUST_ID_COL, right_on=CUST_ID_COL, how='left')
                
                
                # Filter the data according to the choice
                if event == 'All events':
                    y_data = dfc.groupby(by=[dist])[time_interval].value_counts().sort_index()
                else:
                    y_data = dfc[dfc[EVENT_TYPE_COL] == event].groupby(by=[dist])[time_interval].value_counts().sort_index()

                
                trace = []

                for g in sorted(list(dfcust[dist].unique())):
                    if dist == 'age_group':
                        name = str(g) + " - " + str(g + bin_size)
                    else:
                        name = g
                    trace.append(go.Bar(x=x_data, y=y_data.loc[g], name=name))


            else:
                # Filter the data according to the choice
                if event == 'All events':
                    y_data = dfc[time_interval].value_counts().sort_index()
                else:
                    y_data = dfc[dfc[EVENT_TYPE_COL] == event][time_interval].value_counts().sort_index()



            # Create the bar plot
            
                trace = [go.Bar(x=x_data, y=y_data)]

            # Create the layout
            layout = go.Layout(title=f'Events by Time Interval, Event: {event}', xaxis=dict(title=str(time_interval)), yaxis=dict(title="Event Count"), barmode="group")

            # Create the figure
            fig = go.Figure(data=trace, layout=layout)

            if dist:
                file_name = f'{event}_{time_interval}_by_{dist}.pdf'
            else:
                file_name = f'{event}_{time_interval}.pdf'

            pio.write_image(fig, os.path.join(event_path,file_name), format='pdf')

In [ ]:
# Heatmap
plot_name = 'heatmap'
plot_path = os.path.join(folder_path,plot_name)

# Check if the folder already exists before creating it
if not os.path.exists(plot_path):
    os.mkdir(plot_path)

for i in range(len(events)):
    event_x = events[i]
    for j in range(i, len(events)):
        event_y = events[j]
        dfc = df.copy()
        dfc = dfc[[CUST_ID_COL,EVENT_TYPE_COL,EVENT_DESC_COL,EVENT_DATE_COL]]
        # Filter the data according to the two event types chosen
        dfc = dfc[(dfc[EVENT_TYPE_COL] == event_x) | (dfc[EVENT_TYPE_COL] == event_y)]
        
        

        # Pivot the DataFrame to get a binary matrix where rows are clients, columns are events,
        # and values indicate whether a client performed the event or not.
        pivot_df = dfc[[CUST_ID_COL,EVENT_DESC_COL]].pivot_table(index=CUST_ID_COL, columns=EVENT_DESC_COL, aggfunc='size', fill_value=0)

        # Calculate the correlation between different events (i.e., how often they occur together).
        correlation_matrix = pivot_df.corr()

        # Correlation is meaningless when considering the same event on both axes, so fill with NaN value
        if event_x == event_y:
            np.fill_diagonal(correlation_matrix.values, np.nan)

        else:
            x_vals = list(dfc[dfc[EVENT_TYPE_COL] == event_x][EVENT_DESC_COL].unique())
            y_vals = list(dfc[dfc[EVENT_TYPE_COL] == event_y][EVENT_DESC_COL].unique())
            correlation_matrix = correlation_matrix[x_vals]
            correlation_matrix = correlation_matrix.filter(items = y_vals, axis=0)

        # Create the heatmap trace
        trace = go.Heatmap(
        z=correlation_matrix.values,
        x=correlation_matrix.columns,
        y=correlation_matrix.index,
        colorscale='Viridis',)

        # Create the layout
        layout = go.Layout(title='Event Co-occurrence Heatmap', xaxis=dict(title=event_x, tickfont = dict(size = 10)), yaxis=dict(title=event_y,tickfont = dict(size = 5)))

        # Create the figure
        fig = go.Figure(data=[trace], layout=layout)

        file_name = f'{event_x}_{event_y}.pdf'

        pio.write_image(fig, os.path.join(plot_path,file_name), format='pdf')


In [ ]:
# Events by customer
plot_name = 'events_by_customer'
plot_path = os.path.join(folder_path,plot_name)

# Check if the folder already exists before creating it
if not os.path.exists(plot_path):
    os.mkdir(plot_path)
    
    
for event_type in ['All events'] + events:
    
    event_path = os.path.join(plot_path,event_type)

    # Check if the folder already exists before creating it
    if not os.path.exists(event_path):
        os.mkdir(event_path)
    
    for cust in customers[:10]: ## UP TO THE USER TO CHOOSE HOW MANY CUSTOMERS TO SEE
        
        dfc = df.copy()
        dfcopy = dfc

        # Create query string to retrieve customer info for all selected customers
        query_string = CUST_ID_COL + ' == "' + cust + '"'


        dfc = dfc.query(query_string)
        

        # Filter the data according to the user's choice of event 
        if event_type != "All events": 


            dfc = dfc[dfc[EVENT_TYPE_COL] == event_type]

        start = dfcopy[EVENT_DATE_COL].min()
        end = dfcopy[EVENT_DATE_COL].max()
        span = (end.year - start.year) * 12 + (end.month - start.month) + (float((end.day - start.day)/30))


        x_data = list(dfc[EVENT_DESC_COL].unique())



        # Create the bar plot
        trace = []
        custs = [cust]
        for cust in custs:

            # For each customer, retrieve the counts of the given events. Must reindex by the x axis for value alignment
            y_data = list(dfc[dfc[CUST_ID_COL] == cust][EVENT_DESC_COL].value_counts().reindex(list(dfc[EVENT_DESC_COL].unique()), fill_value=0))
            trace.append(go.Bar(x=x_data, y=y_data, name=cust + ": " + str(round(len(dfcopy[(dfcopy[CUST_ID_COL] == cust) & (dfcopy[EVENT_DATE_COL] >= start) & (dfcopy[EVENT_DATE_COL] <= end)]) / span, 2)) + " events per month in given range"))

        # Create the layout
        layout = go.Layout(title=f'Event Distribution by Customer, Event: {event_type}', xaxis=dict(title="Events",tickfont = dict(size = 5)), yaxis=dict(title="Event Count"), barmode="group", showlegend = True)

        # Create the figure
        fig = go.Figure(data=trace, layout=layout)
        
        file_name = f'events_for_{cust}_by_{event_type}.pdf'

        pio.write_image(fig, os.path.join(event_path,file_name), format='pdf')
        

In [22]:
# Number of customers per event
plot_name = 'customer_count_per_event'
plot_path = os.path.join(folder_path,plot_name)

# Check if the folder already exists before creating it
if not os.path.exists(plot_path):
    os.mkdir(plot_path)  

# UP TO THE USER TO CHOOSE THE DATE RANGES
start = df[EVENT_DATE_COL].min()
end = df[EVENT_DATE_COL].max()
middle = start + (end - start) / 2
starts = [start, middle]
ends = [middle, end]

for event_type in events:
    
    event_path = os.path.join(plot_path,event_type)

    # Check if the folder already exists before creating it
    if not os.path.exists(event_path):
        os.mkdir(event_path)
    
    
    for i in range(len(starts)):
        start = starts[i]
        
        for j in range(i, len(ends)):
            end = ends[j]
            
            dfc = df.copy()
            dfc = dfc[[CUST_ID_COL,EVENT_TYPE_COL,EVENT_DESC_COL,EVENT_DATE_COL]]

            if start and end:

                dfc = dfc[(dfc[EVENT_DATE_COL] >= start) & (dfc[EVENT_DATE_COL] <= end)]

            dfc = dfc[dfc[EVENT_TYPE_COL] == event_type]
            

            events_to_check = dfc[EVENT_DESC_COL].unique()
            trace = []
            for event in events_to_check:

                dfc2 = dfc[dfc[EVENT_DESC_COL] == event]
                # Count the occurrences of each unique customer
                customer_counts = dfc2[CUST_ID_COL].value_counts()

                # Count the number of customers who appear once, twice, three times, and so on
                summary = customer_counts.value_counts().sort_index()

                x_data = list(summary.index)
                y_data = list(summary)

                # Create the bar plot
                trace.append(go.Bar(x=x_data, y=y_data, name = event))

            # Create the layout
            layout = go.Layout(title='Customer Count by event, Event: '+event_type+', Start: '+ start.strftime('%Y-%m-%d') + ', End: '+ end.strftime('%Y-%m-%d'), xaxis=dict(title="Number of times event was completed"), yaxis=dict(title="Customer count"), barmode="stack")

            # Create the figure
            fig = go.Figure(data=trace, layout=layout)
            
            fig.update_layout( 
            title_font=dict(size=10),
            legend=dict(font=dict(size=5),
                        orientation = 'h',
                       y = -1,
                       yanchor = 'auto')
            )

            file_name = event_type+'_from_'+start.strftime('%Y_%m_%d')+'_to_'+end.strftime('%Y_%m_%d')+'.pdf'

            pio.write_image(fig, os.path.join(event_path,file_name), format='pdf')
            
           
            # MAKE PLOT WITHOUT BREAKDOWN
            customer_counts = dfc[CUST_ID_COL].value_counts()

            # Count the number of customers who appear once, twice, three times, and so on
            summary = customer_counts.value_counts().sort_index()

            x_data = list(summary.index)
            y_data = list(summary)
            
            # Create the bar plot
            trace = [go.Bar(x=x_data, y=y_data, name = event)]
            
            # Create the layout
            layout = go.Layout(title='Customer Count by event (no breakdown), Event: '+event_type+', Start: '+ start.strftime('%Y-%m-%d') + ', End: '+ end.strftime('%Y-%m-%d'), xaxis=dict(title="Number of times event was completed"), yaxis=dict(title="Customer count"), barmode="stack")

            # Create the figure
            fig = go.Figure(data=trace, layout=layout)
            
            fig.update_layout( 
            title_font=dict(size=10),  # Set the font size for the title text
            )

            file_name = event_type+'_from_'+start.strftime('%Y_%m_%d')+'_to_'+end.strftime('%Y_%m_%d')+'_no_breakdown.pdf'

            pio.write_image(fig, os.path.join(event_path,file_name), format='pdf')

In [ ]:
# Scatter event frequency
plot_name = 'numeric_to_frequency'
plot_path = os.path.join(folder_path,plot_name)

# Check if the folder already exists before creating it
if not os.path.exists(plot_path):
    os.mkdir(plot_path)
    
## UP TO THE USER TO CHOOSE DATE RANGES
start = df[EVENT_DATE_COL].min()
end = df[EVENT_DATE_COL].max()
middle = start + (end - start) / 2
starts = [start, middle]
ends = [middle, end]

for event_type in events:
    
    event_path = os.path.join(plot_path,event_type)

    # Check if the folder already exists before creating it
    if not os.path.exists(event_path):
        os.mkdir(event_path)
        
    for x_axis in splitting_cols_numeric:
    
    
        for i in range(len(starts)):
            start = starts[i]

            for j in range(i, len(ends)):
                end = ends[j]



                dfc = df.copy()

                if start and end:
                    dfc = dfc[(dfc[EVENT_DATE_COL] >= start) & (dfc[EVENT_DATE_COL] <= end)]

                if event_type != 'All events':
                    dfc = dfc[dfc[EVENT_TYPE_COL] == event_type]

                # Copy the cust_info df to retreive customer ages and account ages
                dfcust = df_cust_info
                dfcust = dfcust[[CUST_ID_COL,AGE_COL,START_D_COL, SALARY_COL]]
                dfcust[AGE_COL] = dfcust[AGE_COL].astype(int)


                if x_axis == START_D_COL:

                    dfcust = dfcust.sort_values(by=[START_D_COL], ascending = False)
                    x_data = pd.Series([(datetime.now() - start_date) for start_date in dfcust[START_D_COL]]).dt.total_seconds() / 86400.0

                else:
                    dfcust = dfcust.sort_values(by=[x_axis])
                    x_data = dfcust[x_axis]

                
                # ALWAYS WILL BE HISTOGRAM
                num_bins = 5
                if x_axis == START_D_COL:
                    dfcust['bins'] = pd.cut(pd.Series([(datetime.now() - start_date) for start_date in dfcust[START_D_COL]]).dt.total_seconds() / 86400.0 ,bins = num_bins)
                else:
                    dfcust['bins'] = pd.cut(dfcust[x_axis],bins= num_bins)

                merged_df = dfcust.merge(dfc.groupby(CUST_ID_COL).size().reset_index(name='count'), on=CUST_ID_COL, how='left')
                span = (end.year - start.year) * 12 + (end.month - start.month) + (float((end.day - start.day)/30))

                y_data = merged_df.groupby(by = ['bins'])['count'].mean()
                y_data = [num / span for num in y_data]
                x_data = sorted(list(dfcust['bins'].astype(str).unique()))
                trace = [go.Bar(x = x_data, y = y_data)]

                layout = go.Layout(title='Plot of ' + x_axis + ' vs. Event Frequency, Event: '+event_type+', Start: '+ start.strftime('%Y-%m-%d') + ', End: '+ end.strftime('%Y-%m-%d'),
                                   xaxis=dict(title=x_axis if x_axis != START_D_COL else x_axis + ' (number of days ago)'),
                                   yaxis=dict(title='Event frequency (per month)'),
                                   hovermode='closest')
                
        
                fig = go.Figure(data=trace, layout=layout)
            
                fig.update_layout( 
                    title_font=dict(size=10),  # Set the font size for the title text
                    )


                file_name = event_type+'_from_'+start.strftime('%Y_%m_%d')+'_to_'+end.strftime('%Y_%m_%d')+'_by_'+x_axis+'.pdf'

                pio.write_image(fig, os.path.join(event_path,file_name), format='pdf')